In [24]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from custom_functions import *

## Choosing the best Accelerator for Matrix Decomposition

In [25]:
dsa_accme_df = pd.read_csv('./data/cholesky/accme_dsa_parameters.csv')
scm_accme_df = pd.read_csv('./data/cholesky/accme_scm_parameters.csv')
ambit_accme_df = pd.read_csv('./data/cholesky/accme_ambit_parameters.csv')
upmem_accme_df = pd.read_csv('./data/cholesky/accme_upmem_parameters.csv')

In [26]:
# Accelerator Parameters
dsa_frequency = 1200000000
scm_frequency = 1000000000
ambit_frequency = 1200000000
upmem_frequency = 350000000
device_frequencies = [dsa_frequency,scm_frequency,ambit_frequency,upmem_frequency]

In [27]:
dsa_accme_df = generate_accme_cycles_column(dsa_accme_df)
scm_accme_df = generate_accme_cycles_column(scm_accme_df)
ambit_accme_df = generate_accme_cycles_column(ambit_accme_df)
upmem_accme_df = generate_accme_cycles_column(upmem_accme_df)
accme_dfs = [dsa_accme_df,scm_accme_df,ambit_accme_df,upmem_accme_df]

## Graph  AIO Uncertainty

In [28]:
def graph_cholesky_comparison(input_dfs, frequencies, aio_uncertainty):
    
    # Some Appearence Settings
    names = ['DSA','SCM','AMBIT','UPMEM']
    line_colors = ['blue','green','red','orange']
    fill_colors = ['rgba(0, 0, 255, 0.2)','rgba(0, 0, 255, 0.2)','rgba(255, 0, 0, 0.2)','rgba(250, 200, 0, 0.2)']
    color_opacity = 0.8
    line_thickness = 2
    axis_title_size = 24
    tick_label_size = 16
    legend_text_size = 18

    # Graph Performance Data
    fig = go.Figure()
    for i in range(len(input_dfs)):
        calculate_cycle_uncertainty_from_aio(input_dfs[i], aio_uncertainty, frequencies[i])
        fig.add_trace(go.Scatter(x=input_dfs[i]['Matrix Size'], y=input_dfs[i]['AIOs/Sec Upper'], fill='none', mode='lines', line = dict(color=line_colors[i], width=line_thickness, dash='dash'), opacity=color_opacity, name='DSA', showlegend=False))
        fig.add_trace(go.Scatter(x=input_dfs[i]['Matrix Size'], y=input_dfs[i]['AIOs/Sec Lower'], fill='tonexty', mode='lines', fillcolor=fill_colors[i], line = dict(color=line_colors[i], width=line_thickness, dash='dash'), opacity=color_opacity, name=(names[i]+'\u00B1'+str(aio_uncertainty)+'X AIO Latency')))
        fig.add_trace(go.Scatter(x=input_dfs[i]['Matrix Size'], y=input_dfs[i]['AIOs/Sec'], fill='none', mode='lines', line = dict(color=line_colors[i], width=line_thickness), opacity=color_opacity, name='DSA', showlegend=False))


    # Format Appearence
    fig.add_annotation(xref="paper", yref="paper", x=0.5, y=-0.35, text="Matrix Size", showarrow=False,
        font=dict(
            size=axis_title_size
        ),
    )
    fig.add_annotation(xref="paper", yref="paper", x=-0.17, y=0.5, text="AIOs / Sec", showarrow=False, textangle=-90,
        font=dict(
            size=axis_title_size
        ),
    )
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            entrywidth=200,
            y=1.02,
            xanchor="left",
            x=-0.05,
            itemwidth=30,
            font=dict(size=legend_text_size),
            traceorder='normal'
            ),
        autosize=False,
        width=600,
        height=380,
    )
    fig.update_xaxes(type="log",tickvals=[2**i for i in range(12)])
    fig.update_yaxes(type="log", dtick=1)

    fig.update_layout(
        xaxis=dict(
            tickfont=dict(
                size=tick_label_size  # Set the tick font size for the x-axis
            )
        ),
        yaxis=dict(
            tickfont=dict(
                size=tick_label_size  # Set the tick font size for the y-axis
            )
        )
    )
    fig.show()


graph_cholesky_comparison(input_dfs=accme_dfs, frequencies=device_frequencies, aio_uncertainty=5)